# Neo4j

>[Neo4j](https://neo4j.com/docs/getting-started/) is a graph database management system developed by `Neo4j, Inc`.

>The data elements `Neo4j` stores are nodes, edges connecting them, and attributes of nodes and edges. Described by its developers as an ACID-compliant transactional database with native graph storage and processing, `Neo4j` is available in a non-open-source "community edition" licensed with a modification of the GNU General Public License, with online backup and high availability extensions licensed under a closed-source commercial license. Neo also licenses `Neo4j` with these extensions under closed-source commercial terms.

>This notebook shows how to use LLMs to provide a natural language interface to a graph database you can query with the `Cypher` query language.

>[Cypher](https://en.wikipedia.org/wiki/Cypher_(query_language)) is a declarative graph query language that allows for expressive and efficient data querying in a property graph.


## Setting up

You will need to have a running `Neo4j` instance. One option is to create a [free Neo4j database instance in their Aura cloud service](https://neo4j.com/cloud/platform/aura-graph-database/). You can also run the database locally using the [Neo4j Desktop application](https://neo4j.com/download/), or running a docker container.
You can run a local docker container by running the executing the following script:

```
docker run \
    --name neo4j \
    -p 7474:7474 -p 7687:7687 \
    -d \
    -e NEO4J_AUTH=neo4j/password \
    -e NEO4J_PLUGINS=\[\"apoc\"\]  \
    neo4j:latest
```

If you are using the docker container, you need to wait a couple of second for the database to start.

In [1]:
import os

from dotenv import load_dotenv
load_dotenv()
groq_api_key: str = os.environ["GROQ_API_KEY"]

In [2]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_groq import ChatGroq

llm = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")

In [3]:
graph = Neo4jGraph()

## Seeding the database

Assuming your database is empty, you can populate it using Cypher query language. The following Cypher statement is idempotent, which means the database information will be the same if you run it one or multiple times.

In [4]:
graph.query(
    """
MERGE (m:Movie {name:"Top Gun", runtime: 120})
WITH m
UNWIND ["Tom Cruise", "Val Kilmer", "Anthony Edwards", "Meg Ryan"] AS actor
MERGE (a:Actor {name:actor})
MERGE (a)-[:ACTED_IN]->(m)
"""
)

[]

## Refresh graph schema information
If the schema of database changes, you can refresh the schema information needed to generate Cypher statements.

In [5]:
graph.refresh_schema()

In [6]:
print(graph.schema)

Node properties:
Movie {title: STRING, tagline: STRING, released: INTEGER, id: STRING, imdbRating: FLOAT, runtime: INTEGER, name: STRING}
Person {born: INTEGER, name: STRING}
Database {name: STRING}
Message {name: STRING}
Product {unitPrice: FLOAT, productName: STRING, unitsInStock: INTEGER, productID: STRING, reorderLevel: INTEGER, supplierID: STRING, categoryID: STRING, discontinued: BOOLEAN, unitsOnOrder: INTEGER, quantityPerUnit: STRING}
Supplier {contactTitle: STRING, city: STRING, phone: STRING, address: STRING, companyName: STRING, supplierID: STRING, homePage: STRING, contactName: STRING, region: STRING, country: STRING, postalCode: STRING, fax: STRING}
Genre {name: STRING}
Actor {name: STRING}
Relationship properties:
ACTED_IN {roles: LIST}
REVIEWED {summary: STRING, rating: INTEGER}
The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:PRODUCED]->(:Movie)
(:Person)-[:WROTE]->(:Movie)
(:Person)-[:FOLLOWS]-

## Enhanced schema information
Choosing the enhanced schema version enables the system to automatically scan for example values within the databases and calculate some distribution metrics. For example, if a node property has less than 10 distinct values, we return all possible values in the schema. Otherwise, return only a single example value per node and relationship property.

In [7]:
enhanced_graph = Neo4jGraph(
    enhanced_schema=True,
)
print(enhanced_graph.schema)

Node properties:
- **Movie**
  - `title`: STRING Example: "The Matrix"
  - `tagline`: STRING Example: "Welcome to the Real World"
  - `released`: INTEGER Min: 1964-12-16, Max: 2012
  - `id`: STRING Example: "1"
  - `imdbRating`: FLOAT Min: 2.4, Max: 9.3
  - `runtime`: INTEGER Min: 120, Max: 120
  - `name`: STRING Available options: ['Top Gun']
- **Person**
  - `born`: INTEGER Min: 1929, Max: 1996
  - `name`: STRING Example: "Keanu Reeves"
- **Database**
  - `name`: STRING Available options: ['Neo4j']
- **Message**
  - `name`: STRING Available options: ['Hello World!']
- **Product**
  - `unitPrice`: FLOAT Min: 2.5, Max: 263.5
  - `productName`: STRING Example: "Chai"
  - `unitsInStock`: INTEGER Min: 0, Max: 125
  - `productID`: STRING Example: "1"
  - `reorderLevel`: INTEGER Min: 0, Max: 30
  - `supplierID`: STRING Example: "1"
  - `categoryID`: STRING Available options: ['1', '2', '7', '6', '8', '4', '3', '5']
  - `discontinued`: BOOLEAN 
  - `unitsOnOrder`: INTEGER Min: 0, Max: 100
  

## Querying the graph

We can now use the graph cypher QA chain to ask question of the graph

In [8]:
chain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True
)

In [9]:
chain.invoke({"query": "Who played in Top Gun?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)-[:ACTED_IN]->(p:Person)
WHERE m.title = 'Top Gun'
RETURN p.name
Full Context:
[]

> Finished chain.


{'query': 'Who played in Top Gun?',
 'result': 'I don\'t have information on that topic. However, I can tell you that the main cast of the 1986 film "Top Gun" included Tom Cruise, Kelly McGillis, Val Kilmer, and Anthony Edwards.'}

## Limit the number of results
You can limit the number of results from the Cypher QA Chain using the `top_k` parameter.
The default is 10.

In [10]:
chain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True, top_k=2
)

In [11]:
chain.invoke({"query": "Who played in Top Gun?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)-[:ACTED_IN]->(p:Person)
WHERE m.title = 'Top Gun'
RETURN p.name
Full Context:
[]

> Finished chain.


{'query': 'Who played in Top Gun?',
 'result': 'I don\'t have information on that topic. However, I can tell you that the main cast of the 1986 film "Top Gun" included Tom Cruise, Kelly McGillis, Val Kilmer, and Anthony Edwards.'}

## Return intermediate results
You can return intermediate steps from the Cypher QA Chain using the `return_intermediate_steps` parameter

In [12]:
chain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True, return_intermediate_steps=True
)

In [13]:
result = chain.invoke({"query": "Who played in Top Gun?"})
print(f"Intermediate steps: {result['intermediate_steps']}")
print(f"Final answer: {result['result']}")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)-[:ACTED_IN]->(p:Person)
WHERE m.title = 'Top Gun'
RETURN p.name
Full Context:
[]

> Finished chain.
Intermediate steps: [{'query': "MATCH (m:Movie)-[:ACTED_IN]->(p:Person)\nWHERE m.title = 'Top Gun'\nRETURN p.name"}, {'context': []}]
Final answer: I don't have information on that topic. However, I can tell you that the main cast of the 1986 film "Top Gun" included Tom Cruise, Kelly McGillis, Val Kilmer, and Anthony Edwards.


## Return direct results
You can return direct results from the Cypher QA Chain using the `return_direct` parameter

In [14]:
chain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True, return_direct=True
)

In [15]:
chain.invoke({"query": "Who played in Top Gun?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)-[:ACTED_IN]->(p:Person)
WHERE m.title = 'Top Gun'
RETURN p.name

> Finished chain.


{'query': 'Who played in Top Gun?', 'result': []}

## Add examples in the Cypher generation prompt
You can define the Cypher statement you want the LLM to generate for particular questions

In [16]:
from langchain_core.prompts.prompt import PromptTemplate

CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:
# How many people played in Top Gun?
MATCH (m:Movie {{title:"Top Gun"}})<-[:ACTED_IN]-()
RETURN count(*) AS numberOfActors

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)

In [17]:
chain.invoke({"query": "How many people played in Top Gun?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title:"Top Gun"})<-[:ACTED_IN]-()
RETURN count(*) AS numberOfActors
Full Context:
[{'numberOfActors': 6}]

> Finished chain.


{'query': 'How many people played in Top Gun?',
 'result': 'Six people played in Top Gun.'}

## Use separate LLMs for Cypher and answer generation
You can use the `cypher_llm` and `qa_llm` parameters to define different llms

In [18]:
chain = GraphCypherQAChain.from_llm(
    graph=graph,
    cypher_llm=llm,
    qa_llm=llm,
    verbose=True,
)

In [19]:
chain.invoke({"query": "Who played in Top Gun?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)-[:ACTED_IN]->(p:Person)
WHERE m.title = 'Top Gun'
RETURN p.name
Full Context:
[]

> Finished chain.


{'query': 'Who played in Top Gun?',
 'result': 'I don\'t have information on that topic. However, I can tell you that the main cast of the 1986 film "Top Gun" included Tom Cruise, Kelly McGillis, Val Kilmer, and Anthony Edwards.'}

## Ignore specified node and relationship types

You can use `include_types` or `exclude_types` to ignore parts of the graph schema when generating Cypher statements.

In [20]:
chain = GraphCypherQAChain.from_llm(
    graph=graph,
    cypher_llm=llm,
    qa_llm=llm,
    verbose=True,
    exclude_types=["Movie"],
)

In [21]:
# Inspect graph schema
print(chain.graph_schema)

Node properties are the following:
Person {born: INTEGER, name: STRING},Database {name: STRING},Message {name: STRING},Product {unitPrice: FLOAT, productName: STRING, unitsInStock: INTEGER, productID: STRING, reorderLevel: INTEGER, supplierID: STRING, categoryID: STRING, discontinued: BOOLEAN, unitsOnOrder: INTEGER, quantityPerUnit: STRING},Supplier {contactTitle: STRING, city: STRING, phone: STRING, address: STRING, companyName: STRING, supplierID: STRING, homePage: STRING, contactName: STRING, region: STRING, country: STRING, postalCode: STRING, fax: STRING},Genre {name: STRING},Actor {name: STRING}
Relationship properties are the following:
ACTED_IN {roles: LIST},REVIEWED {summary: STRING, rating: INTEGER}
The relationships are the following:
(:Person)-[:FOLLOWS]->(:Person),(:Database)-[:SAYS]->(:Message)


## Validate generated Cypher statements
You can use the `validate_cypher` parameter to validate and correct relationship directions in generated Cypher statements

In [22]:
chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True,
    validate_cypher=True,
)

In [23]:
chain.invoke({"query": "Who played in Top Gun?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)<-[:ACTED_IN]-(p:Person)
WHERE m.title = 'Top Gun'
RETURN p.name
Full Context:
[{'p.name': 'Val Kilmer'}, {'p.name': 'Meg Ryan'}, {'p.name': 'Tom Skerritt'}, {'p.name': 'Kelly McGillis'}, {'p.name': 'Tom Cruise'}, {'p.name': 'Anthony Edwards'}]

> Finished chain.


{'query': 'Who played in Top Gun?',
 'result': 'Val Kilmer, Meg Ryan, Tom Skerritt, Kelly McGillis, Tom Cruise, and Anthony Edwards played in Top Gun.'}